In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from keras.models import load_model
import os

# Function to perform prediction
def predict_character():
    global model, word_dict

    # Load the selected image
    file_path = filedialog.askopenfilename()

    if file_path:
        img = cv2.imread(file_path)
        img_copy = img.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (400, 440))
        img_copy = cv2.GaussianBlur(img_copy, (7, 7), 0)

        img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
        _, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
        img_final = cv2.resize(img_thresh, (28, 28))
        img_final = np.reshape(img_final, (1, 28, 28, 1))

        img_pred = word_dict[np.argmax(model.predict(img_final))]

        cv2.putText(img, "Prediction: " + img_pred, (20, 410), cv2.FONT_HERSHEY_DUPLEX, 1.3, color=(255, 0, 30))
        cv2.imshow('Handwritten character recognition', img)

# Function to change the model file
def change_model_file():
    global model
    file_path = filedialog.askopenfilename(filetypes=[("Model Files", "*.h5")])

    if file_path:
        model = load_model(file_path)
        model_label.config(text=f"Current Model: {os.path.basename(file_path)}")
        print(f"Loaded custom model from: {file_path}")

# Load the default model (default epoch1)
default_model_path = 'Models/model_hand.h5'
model = load_model(default_model_path)
word_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L',
             12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W',
             23: 'X', 24: 'Y', 25: 'Z'}

# Create the GUI window
root = tk.Tk()
root.title("Handwritten Character Recognition")
root.geometry("600x400")  # Setting initial window size

# Create a consistent style for buttons
button_style = {'width': 20, 'height': 2, 'bg': '#D3D3D3', 'fg': 'black', 'borderwidth': 2, 'relief': 'groove'}

# Create labels and buttons in the GUI
welcome_label = tk.Label(root, text="Handwritten Character Recognition", font=("Arial", 16))
welcome_label.pack(pady=10)

select_image_button = tk.Button(root, text="Select Image", command=predict_character, **button_style)
select_image_button.pack(pady=10)

change_model_button = tk.Button(root, text="Change Model", command=change_model_file, **button_style)
change_model_button.pack(pady=10)

model_label = tk.Label(root, text="Current Model: " + os.path.basename(default_model_path), font=("Arial", 12))
model_label.pack()

root.mainloop()
